**Problem description**: There is a burning house (which is a board) and we must
reach the edge of the board to escape within given steps. Some places in the
house have unique rules which describes from which direction is allowed to enter.

There is an input text file. The first row contains 3 integers, seperated by
spaces: the first int is the max. number of steps allowed to escape. The other
two ints are the rows and cols of the board (house). We can found the board
below the first row.

S means the starting position. U-D-L-R means we can enter that cell from
up-down-left-right. We can enter without restriction into the 0 valued cells
and it is forbidden to enter into the 1 valued cells.

**Personal Note**: the solution works however its not optimal. This solution should
be rewritten with multiprocessing in the future.

In [1]:
import sys


def case_solution(data_in, limit_time, limit_rows,
                  limit_cols, start_row, start_col):
    # these represents our steps (eg. [-1, 0] is "Up")
    dirs = [[-1, 0], [1, 0], [0, -1], [0, 1]]
    # these are the special entrance characters
    # if we go "Up" we can go in from below ("D")
    dirs_inv = ["D", "U", "R", "L"]
    # the position of our character on the grid
    position = [start_row, start_col]
    # the escape time, which has a wrong value at this point
    esc_time = -1
    # list to contain the escape time of escapes
    escapes = []
    # the time spent by moving
    spent_time = 0
    # sanity check for the time limit (div. only 1 because limits are IDs)
    if limit_time > (limit_rows - 1) * (limit_cols - 1):
        limit_time = (limit_rows - 1) * (limit_cols - 1)
    # sanity check for the borders and their neighbours
    has_exit = sanity_border(data_in)
    # check if we already escaped in our current location or not
    if has_exit == False or limit_time <= 0 or limit_rows < 0 or limit_cols < 0:
        pass
    elif (position[0] == 0 or position[0] == limit_rows or 
    position[1] == 0 or position[1] == limit_cols):
        esc_time = 0
    else:
        # if we are not escaped then initiate moving
        (data_in, position, spent_time, escapes) = moving(
            data_in, position, limit_rows, limit_cols,
            dirs, dirs_inv, spent_time, limit_time, escapes, -1)
        # get minimum escape time when returned
        if escapes:
            esc_time = min(escapes)
    if esc_time == -1:
        sys.stdout.write("NOT POSSIBLE")
    else:
        sys.stdout.write(str(esc_time))


def moving(data_in, position, limit_rows, limit_cols,
           dirs, dirs_inv, spent_time, limit_time, escapes, prev_state):
    # goes until the last item of the last list is progressed
    for i in range(4):
        # forbid to move back into the previous cell to avoid timeout
        if ((prev_state >= 0 and (prev_state % 2) == 0 and i == prev_state + 1) or
        (prev_state >= 0 and (prev_state % 2) != 0 and i == prev_state - 1)):
            pass
        else:
            # we move based on loop of the dirs list
            position = [position[0] + dirs[i][0], position[1] + dirs[i][1]]
            # incrementing the spent time
            spent_time += 1
            # get the symbol written in our new position
            symbol = data_in[position[0]][position[1]]
            # check if the movement was possible or not
            if (spent_time <= limit_time and
            (symbol == 0 or symbol == dirs_inv[i])):
                if (position[0] == 0 or position[0] == limit_rows or 
                    position[1] == 0 or position[1] == limit_cols):
                    # add escape time to list if we're escaped
                    escapes.append(spent_time)
                else:
                    # start another movement if we're not escaped
                    (data_in, position, spent_time, escapes) = moving(
                        data_in, position, limit_rows, limit_cols,
                        dirs, dirs_inv, spent_time, limit_time, escapes, i)
            # redo previous actions
            position = [position[0] - dirs[i][0], position[1] - dirs[i][1]]
            spent_time -= 1
            # assign previous state
            prev_state = -1
    # return to the previous step
    return (data_in, position, spent_time, escapes)


def sanity_border(data_in):
    has_exit = False
    exit_values_x1 = [0, "S", "R"]
    exit_values_x2 = [0, "S", "L"]
    exit_values_y1 = [0, "S", "D"]
    exit_values_y2 = [0, "S", "U"]
    for i in range(len(data_in)):
        if has_exit == True:
            break
        # left border
        if data_in[i][0] in exit_values_x1:
            has_exit = True
            break
        # right border
        if data_in[i][-1] in exit_values_x2:
            has_exit = True
            break
        if (i == 0 or i == len(data_in) - 1):
            for j in range(len(data_in[i])):
                # top border
                if i == 0 and data_in[i][j] in exit_values_y1:
                    has_exit = True
                    break
                # bottom border
                if i == 0 and data_in[i][j] in exit_values_y2:
                    has_exit = True
                    break
    return has_exit


if __name__ == "__main__":
    file_name = ""
    file_name = "ext_files\\escape_from_house\\test0.in"
    if file_name:
        file_in = open(file_name, "r")
    else:
        file_in = sys.stdin
    # reading input into nested lists
    data_in = []
    for line, line_data in enumerate(file_in):
        if line == 0:
            limit_time = int(line_data.split(" ")[0])
            num_rows = int(line_data.split(" ")[1])
            num_cols = int(line_data.split(" ")[2])
        else:
            data_in.append([])
            for pos, char in enumerate(line_data.strip()):
                data_in[line - 1].append(char)
                if char == "S":
                    start_row = line - 1
                    start_col = pos
                elif char == "0" or char == "1":
                    data_in[line - 1][pos] = float(char)
    case_solution(data_in, limit_time, num_rows - 1,
                num_cols - 1, start_row, start_col)


2